In [6]:
#!/usr/bin/env python
# =====================================================================
#  STAGE 13 · Sector & Cluster Insights Dashboard     (pipeline v2)
# =====================================================================
"""
Inputs expected in the same run folder (no hard-coded dates):

stage12/Stage12B_PCA_Scores.csv
stage12/Stage12B_ClusterLabels.csv
stage11/  any CSV containing both “rise” and “prob” (or “probs”)
stage03/Stage3_Data_WithRatios.csv

Env-vars honoured
-----------------
SWAN_YEAR   (yyyy)             → which event folder to search
RUN_DIR     (abs path)         → skip search; use this run directly
OUTPUT_ROOT (default outputs_rff)
ID_COL      (default Symbol)
DATE_COL    (default ReportDate)
SECTOR_COL  (default SectorName)
SCORES_CSV, LABELS_CSV, RISE_CSV, STAGE3_CSV → override any input
PLOT=1      → save PNG plots

Outputs (<run>/stage13/)
------------------------
Sector_Kruskal.csv
Cluster_ANOVA.csv
BimodalSectors.csv
SectorSummary.csv
ClusterSummary.csv
QuickSummary.txt
(optional PNGs if PLOT=1)
"""
from __future__ import annotations
import os, logging, warnings, textwrap
from pathlib import Path

import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kruskal, f_oneway
from sklearn.mixture import GaussianMixture

# ────────────────────────────────────────────────────────────────────
warnings.filterwarnings("ignore", category=FutureWarning)
logging.basicConfig(level=logging.INFO,
    format="%(asctime)s | %(levelname)-7s | %(message)s")
log = logging.getLogger("stage13")

# ╔══════════════════════════════════════════════════════════════════╗
# 0 · BASIC CONFIG                                                   #
# ╚══════════════════════════════════════════════════════════════════╝
SWAN_YEAR   = int(os.getenv("SWAN_YEAR", 2008))
PRE_YEAR    = SWAN_YEAR - 1
OUTPUT_ROOT = Path(os.getenv("OUTPUT_ROOT", "outputs_rff")).expanduser()

ID_COL     = os.getenv("ID_COL",     "Symbol").lower()
DATE_COL   = os.getenv("DATE_COL",   "ReportDate").lower()
SECTOR_COL = os.getenv("SECTOR_COL", "SectorName").lower()

# ╔══════════════════════════════════════════════════════════════════╗
# 1 · LOCATE THE RUN DIRECTORY                                       #
# ╚══════════════════════════════════════════════════════════════════╝
if os.getenv("RUN_DIR"):
    RUN_DIR = Path(os.getenv("RUN_DIR")).expanduser()
else:
    event_dir = OUTPUT_ROOT / f"event={SWAN_YEAR}"
    if not event_dir.is_dir():
        raise RuntimeError(f"No directory {event_dir}")
    # pick most-recent run that already has Stage 12 PCA scores
    candidates = sorted(
        event_dir.glob("*/stage12/Stage12B_PCA_Scores.csv"),
        key=lambda p: p.stat().st_mtime,
        reverse=True,
    )
    if not candidates:
        raise RuntimeError("Run Stage 12 first – no PCA scores found.")
    RUN_DIR = candidates[0].parents[1]

STAGE_DIR = RUN_DIR / "stage13"
STAGE_DIR.mkdir(parents=True, exist_ok=True)
log.info("Stage 13 using run folder: %s", RUN_DIR.name)

# ╔══════════════════════════════════════════════════════════════════╗
# 2 · RESOLVE INPUT FILES                                            #
# ╚══════════════════════════════════════════════════════════════════╝
def _resolve(env_var: str, canonical: Path, fallback_kw: list[str] | None = None) -> Path:
    """env override → canonical path → first file matching all keywords."""
    if os.getenv(env_var):
        p = Path(os.getenv(env_var)).expanduser()
        if not p.is_file():
            raise FileNotFoundError(f"{env_var}='{p}' not found")
        return p
    if canonical.is_file():
        return canonical
    if fallback_kw:
        kws = [k.lower() for k in fallback_kw]
        for fp in canonical.parent.glob("*.csv"):
            nm = fp.name.lower()
            if all(k in nm for k in kws):
                return fp
    raise FileNotFoundError(f"{canonical.name} (or {env_var}) not found")

paths = {
    "scores" : _resolve("SCORES_CSV",
                        RUN_DIR / "stage12" / "Stage12B_PCA_Scores.csv"),
    "labels" : _resolve("LABELS_CSV",
                        RUN_DIR / "stage12" / "Stage12B_ClusterLabels.csv"),
    "rise"   : _resolve("RISE_CSV",
                        RUN_DIR / "stage11" / "Stage11_RISE_Probabilities_All.csv",
                        fallback_kw=["rise", "prob"]),
    "stage3" : _resolve("STAGE3_CSV",
                        RUN_DIR / "stage03" / "Stage3_Data_WithRatios.csv"),
}
log.info("Input artefacts:\n%s",
         "\n".join(f"  · {k}: {v.relative_to(RUN_DIR)}" for k,v in paths.items()))

# ╔══════════════════════════════════════════════════════════════════╗
# 3 · LOAD DATA                                                      #
# ╚══════════════════════════════════════════════════════════════════╝
def _ld(fp: Path, parse_dates: bool = False) -> pd.DataFrame:
    df = pd.read_csv(fp, low_memory=False)
    df.columns = df.columns.str.lower().str.strip()
    if parse_dates and DATE_COL in df.columns:
        df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce")
    return df

scores  = _ld(paths["scores"])
labels  = _ld(paths["labels"])
rise    = _ld(paths["rise"],   parse_dates=True)
stage3  = _ld(paths["stage3"], parse_dates=True)

snap = stage3[stage3[DATE_COL].dt.year == PRE_YEAR].copy()
if snap.empty:
    raise RuntimeError(f"No FY-{PRE_YEAR} rows in Stage 3 snapshot")

prob_cols = [c for c in rise.columns if "prob" in c]
if not prob_cols:
    raise RuntimeError("No probability columns in Stage 11 file")

df = (snap[[ID_COL, DATE_COL, SECTOR_COL]]
        .merge(scores, on=ID_COL, how="left")
        .merge(labels, on=ID_COL, how="left")
        .merge(rise[[ID_COL, DATE_COL] + prob_cols],
               on=[ID_COL, DATE_COL], how="left"))

df["mean_rise_prob"] = (df[prob_cols]
                          .replace([np.inf, -np.inf], np.nan)
                          .mean(axis=1, skipna=True))

# ╔══════════════════════════════════════════════════════════════════╗
# 4 · STATISTICAL TESTS & SUMMARIES                                  #
# ╚══════════════════════════════════════════════════════════════════╝
sector_groups  = [g["mean_rise_prob"].dropna().values
                  for _, g in df.groupby(SECTOR_COL) if len(g) >= 5]
kw_s, kw_p = kruskal(*sector_groups) if len(sector_groups) >= 2 else (np.nan, np.nan)

cluster_groups = [g["mean_rise_prob"].dropna().values
                  for _, g in df.groupby("cluster") if len(g) >= 5]
anova_f, anova_p = f_oneway(*cluster_groups) if len(cluster_groups) >= 2 else (np.nan, np.nan)

pd.DataFrame({"KW_stat":[kw_s], "p_value":[kw_p]})\
    .to_csv(STAGE_DIR / "Sector_Kruskal.csv", index=False)
pd.DataFrame({"ANOVA_F":[anova_f], "p_value":[anova_p]})\
    .to_csv(STAGE_DIR / "Cluster_ANOVA.csv", index=False)

# bimodality scan
rows=[]
for sec,g in df.groupby(SECTOR_COL):
    x=g["mean_rise_prob"].dropna().values.reshape(-1,1)
    if len(x)<20: continue
    bic1=GaussianMixture(1,random_state=0).fit(x).bic(x)
    bic2=GaussianMixture(2,random_state=0).fit(x).bic(x)
    if bic1 - bic2 > 10:
        rows.append({"sector":sec,"ΔBIC":round(bic1-bic2,1),"n":len(x)})
bimodal = pd.DataFrame(rows).sort_values("ΔBIC", ascending=False)
bimodal.to_csv(STAGE_DIR / "BimodalSectors.csv", index=False)

sector_sum  = (df.groupby(SECTOR_COL)["mean_rise_prob"]
                 .agg(['count','median','mean','std','min','max'])
                 .round(4).sort_values("median", ascending=False))
cluster_sum = (df.groupby("cluster")["mean_rise_prob"]
                 .agg(['count','median','mean','std','min','max'])
                 .round(4))
sector_sum .to_csv(STAGE_DIR / "SectorSummary.csv")
cluster_sum.to_csv(STAGE_DIR / "ClusterSummary.csv")

# quick summary
quick = textwrap.dedent(f"""
    === STAGE 13 QUICK SUMMARY (FY-{PRE_YEAR}) ===

    Kruskal-Wallis (sector) : stat = {kw_s:8.3f}   p = {kw_p:.3g}
    One-way ANOVA (cluster) :   F = {anova_f:8.3f}   p = {anova_p:.3g}

    Bimodal sectors (ΔBIC > 10)
    ---------------------------
{bimodal.to_string(index=False) if not bimodal.empty else '      (none detected)'}

    Top-5 sectors by median mean_rise_prob
    -------------------------------------
{sector_sum.head(5).to_string()}

    Cluster summary
    ---------------
{cluster_sum.to_string()}
""").strip()
(STAGE_DIR / "QuickSummary.txt").write_text(quick, encoding="utf-8")
print("\n"+quick+"\n")

# ╔══════════════════════════════════════════════════════════════════╗
# 5 · OPTIONAL PLOTS                                                 #
# ╚══════════════════════════════════════════════════════════════════╝
if os.getenv("PLOT", "0") == "1":
    sns.set_style("whitegrid")
    pc_cols = [c for c in df.columns if c.startswith("pc")]

    if len(pc_cols) >= 2:
        plt.figure(figsize=(6,4))
        sns.scatterplot(x=pc_cols[0], y=pc_cols[1],
                        hue=SECTOR_COL, data=df,
                        palette="tab20", s=40, linewidth=.3)
        plt.tight_layout()
        plt.savefig(STAGE_DIR / "PC1_PC2_by_sector.png", dpi=110)

    top_secs = df[SECTOR_COL].value_counts().head(12).index
    plt.figure(figsize=(9,4))
    sns.boxplot(x=SECTOR_COL, y="mean_rise_prob",
                data=df[df[SECTOR_COL].isin(top_secs)])
    plt.xticks(rotation=35, ha="right")
    plt.tight_layout()
    plt.savefig(STAGE_DIR / "RiseProb_by_sector.png", dpi=110)

    plt.figure(figsize=(5,3))
    sns.boxplot(x="cluster", y="mean_rise_prob", data=df)
    plt.tight_layout()
    plt.savefig(STAGE_DIR / "RiseProb_by_cluster.png", dpi=110)

log.info("✓ Stage 13 complete – artefacts in %s", STAGE_DIR)


2025-06-10 16:34:25,819 | INFO    | Stage 13 using run folder: 20250609
2025-06-10 16:34:25,836 | INFO    | Input artefacts:
  · scores: stage12\Stage12B_PCA_Scores.csv
  · labels: stage12\Stage12B_ClusterLabels.csv
  · rise: stage11\11_RISE_Probabilities_All.csv
  · stage3: stage03\Stage3_Data_WithRatios.csv
c:\Users\Jason Pohl\miniconda3\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\Jason Pohl\miniconda3\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "c:\Users\Jason Pohl\miniconda3\lib\subprocess.py", line 493, in run
    with Popen(*popenargs, **kwargs) as process:
  


=== STAGE 13 QUICK SUMMARY (FY-2007) ===

    Kruskal-Wallis (sector) : stat =  348.685   p = 7.57e-69
    One-way ANOVA (cluster) :   F =   26.872   p = 2.66e-07

    Bimodal sectors (ΔBIC > 10)
    ---------------------------
                sector  ΔBIC   n
    Financial Services 110.3 286
Communication Services  10.4  20

    Top-5 sectors by median mean_rise_prob
    -------------------------------------
                        count  median    mean     std     min     max
sectorname                                                           
Financial Services        286  0.8029  0.7924  0.0453  0.6595  0.8421
Consumer Cyclical          40  0.7833  0.7652  0.0522  0.6516  0.8383
Industrials                67  0.7661  0.7593  0.0576  0.5771  0.8441
Consumer Defensive         35  0.7634  0.7543  0.0690  0.5341  0.8427
Communication Services     20  0.7536  0.7570  0.0666  0.6484  0.8481

    Cluster summary
    ---------------
         count  median    mean     std     min     max
